In [1]:
%load_ext nb_black

import os
import sys

sys.path.append(
    os.path.abspath("/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/codebase")
)
import torch.backends.cudnn as cudnn
import random
from datetime import datetime
import time
from collections import OrderedDict
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn.functional import one_hot
import utils

from Explainer.loss_F import loss_fn_kd, entropy_loss
from Explainer.models.Gated_Logic_Net import Gated_Logic_Net
from Explainer.models.explainer import Explainer
from Explainer.models.pi import Pi
from dataset.dataset_cubs import Dataset_cub_for_explainer
from dataset.utils_dataset import get_dataset_with_image_and_attributes
from Explainer.loss_F import loss_fn_kd, entropy_loss, Selective_Distillation_Loss
from Explainer.models.concepts import Conceptizator
import jupyter_notebook.stats_t as stats
import matplotlib.pyplot as plt
import warnings
import sklearn.metrics as metrics

# from ipython_utils import cal_acc, show_results
from ipywidgets import interact, interactive, fixed, interact_manual
from scipy.stats import percentileofscore
from sklearn.metrics import roc_auc_score, accuracy_score

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [2]:
disease = "pneumonia"
path = f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/mimic_cxr/t/lr_0.01_epochs_60_loss_BCE_W_flattening_type_flatten_layer_features_denseblock4/densenet121/{disease}"
test_attr = torch.load(os.path.join(path, "dataset_g", "test_attributes.pt"))
test_proba_concept = torch.load(os.path.join(path, "dataset_g", "test_proba_concepts.pt"))
test_logits_concept = torch.load(os.path.join(path, "dataset_g", "test_logits_concepts.pt"))
test_logits_bb = torch.load(os.path.join(path, "dataset_g", "test_logits_bb.pt"))
test_class_labels = torch.load(os.path.join(path, "dataset_g", "test_class_labels.pt"))

train_attr = torch.load(os.path.join(path, "dataset_g", "train_attributes.pt"))
train_proba_concept = torch.load(os.path.join(path, "dataset_g", "train_proba_concepts.pt"))
train_logits_concept = torch.load(os.path.join(path, "dataset_g", "train_logits_concepts.pt"))
train_logits_bb = torch.load(os.path.join(path, "dataset_g", "train_logits_bb.pt"))
train_class_labels = torch.load(os.path.join(path, "dataset_g", "train_class_labels.pt"))

val_attr = torch.load(os.path.join(path, "dataset_g", "val_attributes.pt"))
val_proba_concept = torch.load(os.path.join(path, "dataset_g", "val_proba_concepts.pt"))
val_logits_concept = torch.load(os.path.join(path, "dataset_g", "val_logits_concepts.pt"))
val_logits_bb = torch.load(os.path.join(path, "dataset_g", "val_logits_bb.pt"))
val_class_labels = torch.load(os.path.join(path, "dataset_g", "val_class_labels.pt"))

print(
    test_attr.size(), test_proba_concept.size(), test_logits_concept.size(),
    test_logits_bb.size(), test_class_labels.size()
)
print()
print(
    train_attr.size(), train_proba_concept.size(), train_logits_concept.size(),
    train_logits_bb.size(), train_class_labels.size()
)
print()
print(
    val_attr.size(), val_proba_concept.size(), val_logits_concept.size(),
    val_logits_bb.size(), val_class_labels.size()
)

with open(os.path.join(path, "MIMIC_test_configs.pkl"), "rb") as input_file:
    args = pickle.load(input_file)

concepts = args.landmark_names_spec + args.abnorm_obs_concepts


torch.Size([24072, 107]) torch.Size([24072, 107]) torch.Size([24072, 107]) torch.Size([48144]) torch.Size([24072, 1])

torch.Size([187548, 107]) torch.Size([187548, 107]) torch.Size([187548, 107]) torch.Size([375096]) torch.Size([187548, 1])

torch.Size([23449, 107]) torch.Size([23449, 107]) torch.Size([23449, 107]) torch.Size([46898]) torch.Size([23449, 1])


<IPython.core.display.Javascript object>

In [8]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
sub_auroc = []
sub_recall = []
all_auroc = []
cnt = 0
for i in range(len(concepts)):
    auroc_val = metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i])
    recall_val = metrics.recall_score(data_GT[:, i], y_pred[:, i])
    all_auroc.append(auroc_val)
    if auroc_val >= 0.65:
        print(f"{concepts[i]}({i}), auroc: {auroc_val}, recall: {recall_val}")
        sub_auroc.append(auroc_val)
        sub_recall.append(recall_val)
        cnt += 1

print(cnt)
np.mean(np.array(sub_auroc)), np.mean(np.array(sub_recall))

left_hilar(1), auroc: 0.7168489777872639, recall: 0.4818181818181818
right_hilar(2), auroc: 0.7016499235415614, recall: 0.43727598566308246
hilar_unspec(3), auroc: 0.7404369941926842, recall: 0.4804560260586319
left_pleural(4), auroc: 0.8113331363550131, recall: 0.6753554502369669
right_pleural(5), auroc: 0.844205517828665, recall: 0.7074527252502781
pleural_unspec(6), auroc: 0.8021465042509425, recall: 0.7409826753763136
heart_size(7), auroc: 0.7330779741424278, recall: 0.6182795698924731
left_diaphragm(9), auroc: 0.7497197141272534, recall: 0.42891566265060244
right_diaphragm(10), auroc: 0.811617653920448, recall: 0.6407185628742516
diaphragm_unspec(11), auroc: 0.720052901126311, recall: 0.5164835164835165
retrocardiac(12), auroc: 0.7435754212670047, recall: 0.6032811334824758
lower_left_lobe(13), auroc: 0.7214500942402027, recall: 0.45985401459854014
upper_left_lobe(14), auroc: 0.7034597422248304, recall: 0.4365079365079365
lower_right_lobe(15), auroc: 0.7131875016475984, recall: 0.

(0.7591573778251801, 0.5514740411925244)

<IPython.core.display.Javascript object>

In [9]:
mask_select = np.array(all_auroc) >= 0.65

torch_mask = torch.from_numpy(mask_select)
torch_mask.sum(), torch_mask.size()

(tensor(96), torch.Size([107]))

<IPython.core.display.Javascript object>

In [4]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
bad_auroc = []
bad_recall = []
cnt = 0
for i in range(len(concepts)):
    auroc_val = metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i])
    recall_val = metrics.recall_score(data_GT[:, i], y_pred[:, i])
    if auroc_val <= 0.7:
        print(f"{concepts[i]}({i}), auroc: {auroc_val}, recall: {recall_val}")
        cnt += 1
        bad_auroc.append(auroc_val)
        bad_recall.append(recall_val)

print(cnt)
np.mean(np.array(sub_auroc)), np.mean(np.array(sub_recall))

trachea(0), auroc: 0.6338102541684264, recall: 0.3137254901960784
heart_border(8), auroc: 0.6262221862511925, recall: 0.20207253886010362
left_lower_lung(18), auroc: 0.6594926634460418, recall: 0.4710941232680363
left_mid_lung(19), auroc: 0.6422545512530895, recall: 0.30456852791878175
left_upper_lung(20), auroc: 0.6781897311242826, recall: 0.30303030303030304
right_mid_lung(24), auroc: 0.6754379607908847, recall: 0.385
left_costophrenic(30), auroc: 0.68008038964865, recall: 0.30275229357798167
right_costophrenic(31), auroc: 0.6686958210093569, recall: 0.3277777777777778
costophrenic_unspec(32), auroc: 0.6600738691407705, recall: 0.41203703703703703
cardiophrenic_sulcus(33), auroc: 0.6065578851816592, recall: 0.10638297872340426
rib(36), auroc: 0.6713160657289852, recall: 0.40987654320987654
clavicle(45), auroc: 0.5720441572658321, recall: 0.2653061224489796
fracture(61), auroc: 0.6639619848824523, recall: 0.3921302578018996
prominence(67), auroc: 0.642757862322522, recall: 0.474576271

(0.7591573778251801, 0.5514740411925244)

<IPython.core.display.Javascript object>

In [5]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
auroc = []
recall = []
for i in range(len(concepts)):
    try:
        auroc.append(metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i]))
        recall.append(metrics.recall_score(data_GT[:, i], y_pred[:, i]))
    except ValueError:
        pass

print(len(auroc))

mean_auroc = np.mean(np.array(auroc))
mean_recall = np.mean(np.array(recall))
mean_auroc, mean_recall

107


(0.7442951286573369, 0.5236328625960476)

<IPython.core.display.Javascript object>

In [10]:
cnt = 0
th = 0.65
concept_select = []

test_select_attr = []
test_select_proba_concept = []
test_select_logits_concept = []


val_select_attr = []
val_select_proba_concept = []
val_select_logits_concept = []


train_select_attr = []
train_select_proba_concept = []
train_select_logits_concept = []


test_select_proba_concept = []
for i in range(len(concepts)):
    if auroc[i] >= th:
        test_select_attr.append(test_attr[:, i])
        test_select_proba_concept.append(test_proba_concept[:, i])
        test_select_logits_concept.append(test_logits_concept[:, i])

        val_select_attr.append(val_attr[:, i])
        val_select_proba_concept.append(val_proba_concept[:, i])
        val_select_logits_concept.append(val_logits_concept[:, i])

        train_select_attr.append(train_attr[:, i])
        train_select_proba_concept.append(train_proba_concept[:, i])
        train_select_logits_concept.append(train_logits_concept[:, i])

        concept_select.append(concepts[i])
        cnt += 1

test_select_attr = torch.stack(test_select_attr, dim=1)
test_select_proba_concept = torch.stack(test_select_proba_concept, dim=1)
test_select_logits_concept = torch.stack(test_select_logits_concept, dim=1)
val_select_attr = torch.stack(val_select_attr, dim=1)
val_select_proba_concept = torch.stack(val_select_proba_concept, dim=1)
val_select_logits_concept = torch.stack(val_select_logits_concept, dim=1)
train_select_attr = torch.stack(train_select_attr, dim=1)
train_select_proba_concept = torch.stack(train_select_proba_concept, dim=1)
train_select_logits_concept = torch.stack(train_select_logits_concept, dim=1)


print("Test sizes: ")
print(test_select_attr.size())
print(test_select_proba_concept.size())
print(test_select_logits_concept.size())
print("Val sizes: ")
print(val_select_attr.size())
print(val_select_proba_concept.size())
print(val_select_logits_concept.size())
print("Train sizes: ")
print(train_select_attr.size())
print(train_select_proba_concept.size())
print(train_select_logits_concept.size())
print(cnt)

Test sizes: 
torch.Size([24072, 96])
torch.Size([24072, 96])
torch.Size([24072, 96])
Val sizes: 
torch.Size([23449, 96])
torch.Size([23449, 96])
torch.Size([23449, 96])
Train sizes: 
torch.Size([187548, 96])
torch.Size([187548, 96])
torch.Size([187548, 96])
96


<IPython.core.display.Javascript object>

In [12]:
metric = "auroc"
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"torch_concepts_mask_{metric}.pt"),
    tensor_to_save=torch_mask,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_{metric}_attributes.pt"),
    tensor_to_save=test_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_proba_{metric}_concepts.pt"),
    tensor_to_save=test_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_logits_{metric}_concepts.pt"),
    tensor_to_save=test_select_logits_concept,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_{metric}_attributes.pt"),
    tensor_to_save=val_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_proba_{metric}_concepts.pt"),
    tensor_to_save=val_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_logits_{metric}_concepts.pt"),
    tensor_to_save=val_select_logits_concept,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_{metric}_attributes.pt"),
    tensor_to_save=train_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_proba_{metric}_concepts.pt"),
    tensor_to_save=train_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_logits_{metric}_concepts.pt"),
    tensor_to_save=train_select_logits_concept,
)

pickle.dump(
    concept_select,
    open(os.path.join(path, "dataset_g", f"selected_concepts_{metric}.pkl"), "wb"),
)
print(os.path.join(path, "dataset_g"))

/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/mimic_cxr/t/lr_0.01_epochs_60_loss_BCE_W_flattening_type_flatten_layer_features_denseblock4/densenet121/pneumonia/dataset_g


<IPython.core.display.Javascript object>

In [13]:
concept_select

['left_hilar',
 'right_hilar',
 'hilar_unspec',
 'left_pleural',
 'right_pleural',
 'pleural_unspec',
 'heart_size',
 'left_diaphragm',
 'right_diaphragm',
 'diaphragm_unspec',
 'retrocardiac',
 'lower_left_lobe',
 'upper_left_lobe',
 'lower_right_lobe',
 'middle_right_lobe',
 'upper_right_lobe',
 'left_lower_lung',
 'left_upper_lung',
 'left_apical_lung',
 'left_lung_unspec',
 'right_lower_lung',
 'right_mid_lung',
 'right_upper_lung',
 'right_apical_lung',
 'right_lung_unspec',
 'lung_apices',
 'lung_bases',
 'left_costophrenic',
 'right_costophrenic',
 'costophrenic_unspec',
 'mediastinal',
 'spine',
 'rib',
 'right_atrium',
 'right_ventricle',
 'aorta',
 'svc',
 'interstitium',
 'parenchymal',
 'cavoatrial_junction',
 'stomach',
 'effusion',
 'opacity',
 'edema',
 'atelectasis',
 'tube',
 'consolidation',
 'process',
 'abnormality',
 'enlarge',
 'tip',
 'low',
 'line',
 'congestion',
 'catheter',
 'cardiomegaly',
 'fracture',
 'air',
 'tortuous',
 'lead',
 'disease',
 'calcificatio

<IPython.core.display.Javascript object>

In [14]:
y_gt_tr = torch.load(os.path.join(path, "dataset_g", f"train_class_labels.pt"))
(y_gt_tr == 1).sum(dim=0), (y_gt_tr == 0).sum(dim=0)

(tensor([6449]), tensor([181099]))

<IPython.core.display.Javascript object>

In [15]:
((y_gt_tr == 1).sum(dim=0) / y_gt_tr.size(0)).item()

0.034385863691568375

<IPython.core.display.Javascript object>